**Fundamentals of Natural Language Processing**


#Negation and Uncertainty Detection Project

**Installing the necessary packages:**

 SpaCy for natural language processing, along with its Spanish and Catalan models. We’ll also install `unidecode` text cleaning

In [ ]:
# Install required packages (execute only if not already installed)
!pip install spacy
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 42.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Install required packages
!pip install spacy unidecode
!python -m spacy download es_core_news_sm
!python -m spacy download ca_core_news_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 6.0 MB/s eta 0:00:00
  Using cached https://github.com/explosion/spacy-models/releases/download/es_core_news_sm-3.8.0/es_core_news_sm-3.8.0-py3-none-any.whl (12.9 MB)
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 54.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ca_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


### Testing on the Test Set
We use the **`normalize_text`** function for text preprocessing.

Pattern detection is improved by adding helper functions like **`detect_multiword_pattern`** to identify composed negation phrases. We also integrate four rule types into the **`analyze_medical_context`** function: prefix, postfix, UMLS_Term + Negation_Phrase, and Negation_Phrase + UMLS_Term.

All detected matches are logged, counters are updated, and relevant examples are stored for the final output.







**Importing libraries for text processing, NLP, and file handling in Colab**

In [ ]:
import json
import re
import unicodedata
from collections import Counter, defaultdict
import spacy
from google.colab import files
from spacy.matcher import PhraseMatcher
nlp = spacy.load("es_core_news_sm")


**Text normalization:**

Before starting any analysis, we define a function to normalize the clinical text, removing special characters, extra spaces, and anything that might interfere with proper analysis.

In [ ]:
def normalize_text(text):
    """Normalize medical text with Spanish/Catalan character support."""
    text = text.lower()
    text = unicodedata.normalize('NFC', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\*+', '', text)
    # Permitir el apóstrofe (')
    text = re.sub(r"[^\w\s.,;:!?'\-àáèéìíòóùúüñç]", '', text)
    return text
def normalize_for_eval(text):
    text = text.lower()
    text = unicodedata.normalize('NFC', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\*+', '', text)
    text = re.sub(r"[^\w\s.,;:!?'\-àáèéìíòóùúüñç]", '', text)
    return text

Here we will define the key elements our system will use to detect negation and uncertainty:  
- A list of negation and uncertainty cues (including multi-word expressions)  
- A base list of UMLS medical terms.  
- A set of common medical suffixes to help catch additional terms not explicitly listed

In [ ]:

# Define lists of cues and UMLS medical terms
NEGATION_WORDS = [
    "no", "sin", "ausencia de", "descarta", "descartado", "excluye", "excluido", "niega", "negado",
    "negativa", "negación", "ningún", "ninguna", "ninguno", "imposible", "inhallable", "carece de", "nunca",
    "jamás", "tampoco", "ni", "nada", "negativo", "mai",
    "sin evidencia de", "no se observa", "no presenta", "no muestra", "no evidencia", "no compatible con",
    "no concluyente", "no parece", "no se detecta", "sin signos de", "sin síntomas de", "sin indicios de",
    "sin hallazgos de", "sin pruebas de", "sin rastro de", "ausente", "no encontrado", "sin cambios",
    "no se aprecian", "no se ven", "descartando", "descartable", "no hay evidencia de", "no hay indicación de",
    "libre de", "exento de", "sin manifestaciones de", "se excluye", "queda descartado", "ninguna evidencia de",
    "ningún signo de", "sin afección", "no identificado", "negado por el paciente", "negado clínicamente",
    "sin enfermedad", "sin afectación", "no afectado", "no positivo", "resultado negativo",
    "resultado no reactivo", "resultado no positivo"
]

UNCERTAINTY_WORDS = [
    "posible", "quizás", "podría", "sospecha de", "considera", "probable", "aparentemente", "puede", "posiblemente",
    "parece", "se considera", "indeterminado", "probabilidad de", "no concluyente", "eventual", "en estudio",
    "pendiente de evaluación", "sugestivo de", "sugiere", "indica que", "se sospecha de", "podría indicar",
    "dudoso", "no definido", "no específico", "no determinado", "valor incierto", "no claro", "no seguro",
    "compatible con", "aparenta ser", "tendría que evaluarse", "a determinar", "probabilidad baja de",
    "probabilidad alta de", "sin certeza", "hipotético", "hipotéticamente", "a confirmar", "falta de certeza",
    "en posible relación con", "estaría asociado", "aparentemente relacionado con", "se intuye", "se deduce que",
    "en consideració", "probablemente", "tal vez", "aproximadamente"
]

# Global list of UMLS medical terms (initially provided, without terms ending in a medical suffix)
UMLS_MEDICAL_TERMS = [
    "interna", "cistoscopia", "uretra", "cronica", "insuficiencia", "renal", "bloqueo", "auriculoventricular",
    "primer grado", "segundo grado", "hipertension", "arterial", "protesis", "cadera", "herniorrafia", "parto",
    "eutocico", "rotura", "membranas", "prematuro", "lactancia materna", "aguda", "parcial", "angiomiolipoma",
    "quistes", "renales", "fractura", "mandibular", "ictus", "infarto", "isquemico", "cerebral", "endovenosa",
    "microcirugia", "endolaringea", "sensitiva", "axonal", "multifactorial", "déficit"
]

# List of medical suffixes
MEDICAL_SUFFIXES = [
    "nosis", "tomia", "patia", "losis", "lisis", "iasis", "scopica", "scopia", "tocico", "itis",
    "algia", "oma", "emia", "plegia", "penia", "cele", "plasia", "ectasia", "uria"
]

# === Build PhraseMatchers ===
neg_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
neg_patterns = [nlp.make_doc(cue) for cue in NEGATION_WORDS]
neg_matcher.add("NEGATION", neg_patterns)

unc_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
unc_patterns = [nlp.make_doc(cue) for cue in UNCERTAINTY_WORDS]
unc_matcher.add("UNCERTAINTY", unc_patterns)

**Helper functions for detection and evaluation**  
Next, we define our three utility functions:  
1. One to detect double negation in a context window.  
2. One to extract ground truth labels (`NEG` and `UNC`) from the annotated dataset.  
3. And one to identify terms based on typical medical suffixes.




In [ ]:

def is_double_negation(tokens):
    """Check if a sequence of tokens contains double negation."""
    simple_negation_cues = {"no", "sin", "nunca", "jamás", "ningún", "ninguna", "nadie", "ninguno", "negado", "niega"}
    negation_count = sum(1 for token in tokens if token in simple_negation_cues)
    return negation_count >= 2

def extract_ground_truth(record, record_index):
    gt_neg = set()
    gt_unc = set()

    entries = []
    if record.get("annotations"):
        entries = record["annotations"]
    elif record.get("predictions"):
        entries = record["predictions"]

    for ann in entries:
        for res in ann.get("result", []):
            labels = res.get("value", {}).get("labels", [])
            start = res.get("value", {}).get("start", None)
            end = res.get("value", {}).get("end", None)
            text_val = res.get("value", {}).get("text", "").lower()
            if start is not None and end is not None:
                if "NEG" in labels:
                    gt_neg.add((record_index, start, end, text_val))
                if "UNC" in labels:
                    gt_unc.add((record_index, start, end, text_val))
    return gt_neg, gt_unc


def detect_medical_suffix(word):
    """Return True if the word ends with any of the defined medical suffixes."""
    for suffix in MEDICAL_SUFFIXES:
        if word.endswith(suffix):
            return True
    return False


**Main analysis Function**  
This function willbring everything together. It will go through each sentence, looking for negation or uncertainty cues, and analyzing the surrounding context to detect relevant medical terms either from the UMLS list or by matching medical suffixes.

It also groups detections by context, flags double negation when found, and keeps track of everything for evaluation later on.


In [ ]:


# Normalize UMLS medical terms to lowercase lemmas
MEDICAL_LEMMA_SET = set([term.lower() for term in UMLS_MEDICAL_TERMS])

def analyze_medical_context(text, nlp, neg_matcher, unc_matcher):
    """
    Uses PhraseMatchers to detect negation and uncertainty cues in context windows.
    Expands with bigram medical term detection and increased context window.
    """
    normalized_text = normalize_text(text)
    doc = nlp(normalized_text)
    results = {
        "negated_terms_grouped": [],
        "uncertain_terms_grouped": [],
        "double_negated_terms": [],
        "negation_cues_used": Counter(),
        "uncertainty_cues_used": Counter(),
        "medical_terms_found": Counter(),
        "medical_suffix_terms": []
    }

    for sent in doc.sents:
        tokens = list(sent)
        lemmas = [token.lemma_.lower() for token in tokens]

        # NEGATION MATCHER
        for match_id, start, end in neg_matcher(sent):
            cue = sent[start:end].text.lower()
            window_start = max(0, start - 15)
            window_end = min(len(tokens), end + 15)
            context = tokens[window_start:window_end]
            context_text = " ".join([t.text for t in context])
            detections = []

            for token in context:
                term = token.lemma_.lower()
                if token.is_stop or token.like_num:
                    continue
                if term in MEDICAL_LEMMA_SET or detect_medical_suffix(term):
                    detection = {
                        "term": term,
                        "detected_text": token.text,
                        "start": token.idx,
                        "end": token.idx + len(token.text)
                    }
                    detections.append(detection)
                    results["medical_terms_found"][term] += 1
                    if detect_medical_suffix(term) and term not in MEDICAL_LEMMA_SET:
                        results["medical_suffix_terms"].append(detection)

            # BIGRAM DETECTION
            for j in range(len(context) - 1):
                bigram = f"{context[j].text.lower()} {context[j+1].text.lower()}"
                if bigram in MEDICAL_LEMMA_SET:
                    detection = {
                        "term": bigram,
                        "detected_text": f"{context[j].text} {context[j+1].text}",
                        "start": context[j].idx,
                        "end": context[j+1].idx + len(context[j+1].text)
                    }
                    detections.append(detection)
                    results["medical_terms_found"][bigram] += 1

            if detections:
                group = {
                    "cue": cue,
                    "context": context_text,
                    "window_start": context[0].idx,
                    "window_end": context[-1].idx + len(context[-1].text),
                    "detections": detections
                }
                if is_double_negation(lemmas[window_start:window_end]):
                    results["double_negated_terms"].append(group)
                else:
                    results["negated_terms_grouped"].append(group)
                results["negation_cues_used"][cue] += len(detections)

        # UNCERTAINTY MATCHER
        for match_id, start, end in unc_matcher(sent):
            cue = sent[start:end].text.lower()
            window_start = max(0, start - 15)
            window_end = min(len(tokens), end + 15)
            context = tokens[window_start:window_end]
            context_text = " ".join([t.text for t in context])
            detections = []

            for token in context:
                term = token.lemma_.lower()
                if token.is_stop or token.like_num:
                    continue
                if term in MEDICAL_LEMMA_SET or detect_medical_suffix(term):
                    detection = {
                        "term": term,
                        "detected_text": token.text,
                        "start": token.idx,
                        "end": token.idx + len(token.text)
                    }
                    detections.append(detection)
                    results["medical_terms_found"][term] += 1

            # BIGRAM DETECTION
            for j in range(len(context) - 1):
                bigram = f"{context[j].text.lower()} {context[j+1].text.lower()}"
                if bigram in MEDICAL_LEMMA_SET:
                    detection = {
                        "term": bigram,
                        "detected_text": f"{context[j].text} {context[j+1].text}",
                        "start": context[j].idx,
                        "end": context[j+1].idx + len(context[j+1].text)
                    }
                    detections.append(detection)
                    results["medical_terms_found"][bigram] += 1

            if detections:
                group = {
                    "cue": cue,
                    "context": context_text,
                    "window_start": context[0].idx,
                    "window_end": context[-1].idx + len(context[-1].text),
                    "detections": detections
                }
                results["uncertainty_cues_used"][cue] += len(detections)
                results["uncertain_terms_grouped"].append(group)

    return results


**Evaluation metrics:**

We will now impkement a function to calculate precision, recall, and F1 score. Which are basic but essential metrics to evaluate how well our system detects negation and uncertainty compared to the annotated ground truth

In [ ]:
def compute_metrics(predicted, ground_truth):
    tp = len(predicted & ground_truth)
    fp = len(predicted - ground_truth)
    fn = len(ground_truth - predicted)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
    jaccard = tp / len(predicted | ground_truth) if (predicted | ground_truth) else 0.0

    return precision, recall, f1, jaccard


**Final execution  evaluation and export**  
Great! Now we that we have everything we need. Lets implement the main function that will run the entire pipeline on the test set. It loads the model and data, processes each record, applies the detection logic, and updates statistics and evaluation metrics for negation and uncertainty.

It also prints out cue frequencies, example detections, and performance results (precision, recall, F1, and Jaccard accuracy).  
At the end, it saves the structured output to a JSON file and makes it available for download.


In [ ]:
def build_phrase_matcher(nlp, cue_list):
    matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
    patterns = [nlp.make_doc(cue) for cue in cue_list]
    matcher.add("CUES", patterns)
    return matcher

def main():
    import time
    print("Loading SpaCy model...")
    nlp = spacy.load("es_core_news_sm")
    neg_matcher = build_phrase_matcher(nlp, NEGATION_WORDS)
    unc_matcher = build_phrase_matcher(nlp, UNCERTAINTY_WORDS)

    print("Please upload your JSON file...")
    uploaded = files.upload()
    filename = next(iter(uploaded))
    with open(filename, 'r', encoding='utf-8') as file:
        data = json.load(file)

    total_negation_counter = Counter()
    total_uncertainty_counter = Counter()
    total_medical_counter = Counter()
    negated_examples = []
    uncertain_examples = []

    # Conjuntos para evaluación: (record_index, start, end, text)
    all_predicted_neg = set()
    all_ground_truth_neg = set()
    all_predicted_unc = set()
    all_ground_truth_unc = set()

    records_with_neg = []
    records_with_unc = []
    output_data = []

    start_time = time.time()
    print(f"Processing {len(data)} records...")
    for i, record in enumerate(data):
        if i % 10 == 0:
            print(f"Processing record {i+1}/{len(data)}...")
        text = record.get("data", {}).get("text", "")
        if not text:
            continue

        results = analyze_medical_context(text, nlp, neg_matcher, unc_matcher)
        total_negation_counter.update(results["negation_cues_used"])
        total_uncertainty_counter.update(results["uncertainty_cues_used"])
        total_medical_counter.update(results["medical_terms_found"])

        if results.get("negated_terms_grouped"):
            records_with_neg.append(i)
        if results.get("uncertain_terms_grouped"):
            records_with_unc.append(i)

        # Procesar detecciones agrupadas para negación
        if "negated_terms_grouped" in results:
            for group in results["negated_terms_grouped"]:
                terms_info = "; ".join([f"{d['term']} (detected: {d['detected_text']})" for d in group["detections"]])
                if len(negated_examples) < 20:
                    negated_examples.append(f"Record {i}: Context: '{group['context']}' (cue: {group['cue']}, terms: {terms_info}, window: {group['window_start']}-{group['window_end']})")
                    # Usar el contexto agrupado como clave para evaluación
                    normalized_cue = normalize_for_eval(group["cue"])
                    all_predicted_neg.add((i, normalized_cue))

        # Procesar detecciones agrupadas para incertidumbre
        if "uncertain_terms_grouped" in results:
            for group in results["uncertain_terms_grouped"]:
                terms_info = "; ".join([f"{d['term']} (detected: {d['detected_text']})" for d in group["detections"]])
                if len(uncertain_examples) < 20:
                    uncertain_examples.append(f"Record {i}: Context: '{group['context']}' (cue: {group['cue']}, terms: {terms_info}, window: {group['window_start']}-{group['window_end']})")
                    normalized_cue = normalize_for_eval(group["cue"])
                    all_predicted_unc.add((i, normalized_cue))

        # Extraer ground truth (NEG y UNC) desde las anotaciones o predicciones en el JSON
        gt_neg, gt_unc = extract_ground_truth(record, i)
        all_ground_truth_neg.update(gt_neg)
        all_ground_truth_unc.update(gt_unc)

        output_record = {
            "id": record.get("id", None),
            "record_index": i,
            "predicted": {
                "negated_terms_grouped": results.get("negated_terms_grouped", []),
                "uncertain_terms_grouped": results.get("uncertain_terms_grouped", []),
                "double_negated_terms": results.get("double_negated_terms", []),
                "medical_suffix_terms": results.get("medical_suffix_terms", [])
            }
        }
        output_data.append(output_record)

    end_time = time.time()
    processing_time = end_time - start_time
    print(" Ground truth samples:")
    for i, item in enumerate(sorted(all_ground_truth_neg)[:3]):
        print(f"GT: {item}")

    print("\n Predicted samples:")
    for i, item in enumerate(sorted(all_predicted_neg)[:3]):
        print(f"PRED: {item}")

    print("\nINTERSECTION:")
    print(all_predicted_neg & all_ground_truth_neg)


    # Now compute metrics as usual
    prec_neg, rec_neg, f1_neg, acc_neg = compute_metrics(all_predicted_neg, all_ground_truth_neg)

    # Calcular métricas para negación, incertidumbre y combinadas
    prec_neg, rec_neg, f1_neg, acc_neg = compute_metrics_loose_match(all_predicted_neg, all_ground_truth_neg)
    prec_unc, rec_unc, f1_unc, acc_unc = compute_metrics_loose_match(all_predicted_unc, all_ground_truth_unc)

    combined_predicted = all_predicted_neg.union(all_predicted_unc)
    combined_ground_truth = all_ground_truth_neg.union(all_ground_truth_unc)
    prec_comb, rec_comb, f1_comb, acc_comb = compute_metrics_loose_match(combined_predicted, combined_ground_truth)


    print("\n=== STATISTICS ===")
    print("\nNegation Cue Frequencies (affecting medical terms):")
    for word, freq in total_negation_counter.most_common():
        print(f"{word}: {freq}")
    print("\nUncertainty Cue Frequencies (affecting medical terms):")
    for word, freq in total_uncertainty_counter.most_common():
        print(f"{word}: {freq}")
    print("\nMedical Terms Frequencies:")
    for word, freq in total_medical_counter.most_common():
        print(f"{word}: {freq}")

    print("\n=== EXAMPLES OF NEGATED MEDICAL TERMS ===")
    for example in negated_examples:
        print(example)
    print("\n=== EXAMPLES OF UNCERTAIN MEDICAL TERMS ===")
    for example in uncertain_examples:
        print(example)

    print("\n=== TOTAL DETECTED CONTEXTS ===")
    print(f"Total detected negated contexts: {len(all_predicted_neg)}")
    print(f"Total detected uncertain contexts: {len(all_predicted_unc)}")
    print(f"Total combined detected contexts: {len(combined_predicted)}")

    print("\n=== RECORDS WITH DETECTIONS ===")
    print(f"Records with negation detections: {records_with_neg}")
    print(f"Records with uncertainty detections: {records_with_unc}")

    print("\n=== EFFICIENCY ===")
    print(f"Total processing time: {processing_time:.2f} seconds")

    print("\n=== EVALUATION METRICS ===")
    print("\nNegation Metrics:")
    print(f"Precision: {prec_neg:.2f}, Recall: {rec_neg:.2f}, F1: {f1_neg:.2f}, Accuracy (Jaccard): {acc_neg:.2f}")
    print(f"\n Matches found using cue containment! TP: {int(prec_comb * (len(combined_predicted)))}")

    print("\nUncertainty Metrics:")
    print(f"Precision: {prec_unc:.2f}, Recall: {rec_unc:.2f}, F1: {f1_unc:.2f}, Accuracy (Jaccard): {acc_unc:.2f}")
    print(f"Matched uncertainty cues (TP estimate): {int(prec_unc * len(all_predicted_unc))}")

    print("\nCombined (Negation + Uncertainty) Metrics:")
    print(f"Precision: {prec_comb:.2f}, Recall: {rec_comb:.2f}, F1: {f1_comb:.2f}, Accuracy (Jaccard): {acc_comb:.2f}")
    print(f"Matched total cues (TP estimate): {int(prec_comb * len(combined_predicted))}")

    output_filename = "output.json"
    with open(output_filename, "w", encoding="utf-8") as outfile:
        json.dump(output_data, outfile, ensure_ascii=False, indent=2)
    print(f"\nOutput file '{output_filename}' generated.")
    files.download(output_filename)

if __name__ == "__main__":
    main()


Loading SpaCy model...
Please upload your JSON file...


Saving negacio_test_v2024.json to negacio_test_v2024 (1).json
Processing 64 records...
Processing record 1/64...
Processing record 11/64...
Processing record 21/64...
Processing record 31/64...
Processing record 41/64...
Processing record 51/64...
Processing record 61/64...
 Ground truth samples:
GT: (0, 395, 398, '')
GT: (0, 499, 505, '')
GT: (0, 1111, 1119, '')

 Predicted samples:
PRED: (0, 'niega')
PRED: (0, 'no')
PRED: (10, 'niega')

INTERSECTION:
set()

=== STATISTICS ===

Negation Cue Frequencies (affecting medical terms):
no: 26
niega: 8
ni: 2
sin: 2

Uncertainty Cue Frequencies (affecting medical terms):
probable: 2
sospecha de: 1

Medical Terms Frequencies:
parto: 14
apendicectomia: 4
renal: 3
neoplasia: 3
arterial: 3
protesis: 2
adenoidectomia: 2
hipertension: 2
colecistectomia: 2
colecistitis: 1
prematuro: 1
ictus: 1
fractura: 1
mandibular: 1
ureterorrenoscopia: 1

=== EXAMPLES OF NEGATED MEDICAL TERMS ===
Record 0: Context: 'motiu d'ingres induccion al parto por pequeño pa

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>